Importing the libraries

In [1]:
import sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import numpy as np
import keras

C:\Users\Rens\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Read data file

In [2]:
df2 = pd.read_csv("out_without_nan_mood_target.csv")

Mood back to whole numbers

In [3]:
df=df2
df2['mood_mean_TARGET'] = df2['mood_mean_TARGET'].astype(float)
dfNonNan = df2.dropna(subset = ['mood_mean_TARGET'] )
dfNonNan = pd.concat([dfNonNan,pd.get_dummies(dfNonNan['id'])],axis=1)
df['mood_mean_TARGET'] = df['mood_mean_TARGET'].astype(float).round().astype(str)
df = pd.concat([df,pd.get_dummies(df['id'])],axis=1)

Only select columns with high correlation

In [10]:
corr=dfNonNan.corr()
goodColumns= corr[abs(corr['mood_mean_TARGET'])>0.10]['mood_mean_TARGET'].sort_values()
goodColumns = goodColumns.drop(['mood_mean_TARGET'])
print(goodColumns)

AS14.07                           -0.264961
AS14.12                           -0.195867
weekDay_time_5                    -0.107756
AS14.33                           -0.107082
AS14.27                            0.100447
appCat.entertainment_sum_time_1    0.104142
appCat.entertainment_sum           0.104142
appCat.other_sum_time_4            0.105976
appCat.game_sum_time_3             0.108747
activity_mean_time_3               0.109724
appCat.entertainment_sum_time_3    0.112453
activity_mean_time_1               0.112584
activity_mean                      0.112584
activity_mean_time_2               0.119888
appCat.other_sum_time_5            0.121189
appCat.game_sum_time_4             0.122164
appCat.game_sum_time_5             0.123017
appCat.entertainment_sum_time_4    0.123766
appCat.entertainment_sum_time_5    0.130096
AS14.32                            0.132730
AS14.29                            0.142908
AS14.03                            0.163754
AS14.30                         

Split in train and test set

In [11]:
notRelevantAll=df.drop(['id','date'],axis=1).dropna()
goldY= notRelevantAll['mood_mean_TARGET']
relevant = notRelevantAll.drop(['mood_mean_TARGET'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(relevant[goodColumns.keys()], goldY, test_size=0.33, random_state=42)
y_train_cat = keras.utils.to_categorical(y_train.astype(float).astype(int))
y_test_cat = keras.utils.to_categorical(y_test.astype(float).astype(int))

goldY_train = keras.utils.to_categorical(goldY.astype(float).astype(int))

Setup model

In [12]:
model = Sequential()
model.add(Dense(len(X_test.columns), activation='relu', input_shape=(len(X_test.columns),)))
model.add(Dense(8, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 35)                1260      
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 288       
_________________________________________________________________
dense_6 (Dense)              (None, 10)                90        
Total params: 1,638
Trainable params: 1,638
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
                   
model.fit(X_train, y_train_cat,epochs=100, batch_size=1, verbose=1)

Epoch 1/100
814/814 [==============================] - 1s 1ms/step - loss: 1.3505 - acc: 0.4533
Epoch 2/100
814/814 [==============================] - 1s 705us/step - loss: 1.1616 - acc: 0.5061
Epoch 3/100
814/814 [==============================] - 1s 714us/step - loss: 1.1101 - acc: 0.5209
Epoch 4/100
814/814 [==============================] - 1s 682us/step - loss: 1.0863 - acc: 0.5467
Epoch 5/100
814/814 [==============================] - 1s 753us/step - loss: 1.0629 - acc: 0.5663
Epoch 6/100
814/814 [==============================] - 1s 788us/step - loss: 1.0491 - acc: 0.5614
Epoch 7/100
814/814 [==============================] - 1s 792us/step - loss: 1.0416 - acc: 0.5627
Epoch 8/100
814/814 [==============================] - 1s 679us/step - loss: 1.0360 - acc: 0.5676
Epoch 9/100
814/814 [==============================] - 1s 714us/step - loss: 1.0240 - acc: 0.5627
Epoch 10/100
814/814 [==============================] - 1s 675us/step - loss: 1.0206 - acc: 0.5774
Epoch 11/100
814/814 

In [15]:
test_loss, test_acc = model.evaluate(X_test, y_test_cat)

print('Test accuracy:', test_acc)

402/402 [==============================] - 0s 131us/step
Test accuracy: 0.6044776125333795
